In [1]:
#coding:utf-8

# ライブラリのダウンロード
import requests
from bs4 import BeautifulSoup
# import urllib
import urllib.request
import sys
import zipfile
import csv
import os
import pyprind
from pathlib import Path
import shutil
import pandas as pd
import re

In [2]:
# 変数
csvUrl= "https://www.aozora.gr.jp/index_pages/list_person_all_extended_utf8.zip"
csvFileName = "list_person_all_extended_utf8"
autherName=["芥川竜之介","太宰治","夏目漱石","宮沢賢治"]
autherID = ["000879","000035","000148","000081"]
dataPath = "aozora"
trainPath = "train"
testPath = "test"

In [3]:
# 青空文庫の作者情報csvをダウンロード(Zip)
urllib.request.urlretrieve(csvUrl,csvFileName+".zip")

('list_person_all_extended_utf8.zip',
 <http.client.HTTPMessage at 0x1c6b63e5e48>)

In [4]:
# Zipを展開する
with zipfile.ZipFile(csvFileName+".zip") as existing_zip:
    existing_zip.extractall('./')

In [5]:
# csvを読み込む
csvReader = ""
with open("./"+csvFileName+".csv",encoding='utf-8') as f:
    reader = csv.reader(f)
    csvReader = [row for row in reader]

In [6]:
# 保存するフォルダーの作成
if not os.path.exists(dataPath):
    os.mkdir(dataPath)
for auther in autherName:
    if not os.path.exists(dataPath+"/"+auther):
        os.mkdir(dataPath+"/"+auther)

In [7]:
# 青空文庫から作品をまとめてダウンロードするので時間がかかります！
humanIdOrder = csvReader[0].index("人物ID")
textFileOrder = csvReader[0].index("XHTML/HTMLファイルURL")
workNameOrder = csvReader[0].index("作品名")
roleFlag = csvReader[0].index("役割フラグ")
pbar = pyprind.ProgBar(len(csvReader))

# 青空文庫のファイルを保存する
def saveAozoraFile(auther,work,url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'lxml')
    elem = soup.find(class_='main_text')
    if elem==None:
        return
    rp_all = elem.find_all('rp')
    for rp in rp_all:
        rp.extract()
    rt_all = elem.find_all('rt')
    for rt in rt_all:
        rt.extract()

    text = elem.get_text()
    with open(dataPath+"/"+auther+"/"+work+".txt","w",encoding='utf-8')as f:
        f.write(text)

for row in csvReader:
    pbar.update()
    if autherID.count(row[humanIdOrder])!=0 and row[roleFlag]=="著者" and re.match("https?://www.aozora.gr.jp/cards/",row[textFileOrder]) and "_" in row[textFileOrder]:
        selectAuther = autherID.index(row[humanIdOrder])
        if not os.path.exists(dataPath+"/"+autherName[selectAuther]+"/"+row[workNameOrder]+".txt"):
            saveAozoraFile(autherName[selectAuther],row[workNameOrder],row[textFileOrder])

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:19


In [8]:
# トレーニングデータを読み込む
df = pd.DataFrame()
# testとtrainのフォルダを探索する
for s in ([dataPath]):
    #  posとnegのフォルダを探索する
    for l,i in zip(autherName,range(len(autherName))):
        # フォルダ名を結合する
        path = os.path.join("./", s, l)
        # ファイルを開いてpandasのDataFrameに追加する
        for j,file in enumerate(sorted(os.listdir(path))):
            if j>=100:
                break
            with open(os.path.join(path, file),'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, i]],ignore_index=True)
df.columns = ['ドキュメント', '作者ID']


In [9]:
# シャッフルする
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))

In [11]:
# 読み込めているかチェック
print(df.shape)
df.head(10)

(400, 2)


,ドキュメント,作者ID
132,\nはしがき\n\n\n もの思う葦という題名にて、日本浪曼派の機関雑誌におよそ一箇年ほどつ...,1
309,\n\nこのみちの醸すがごとく\n\n粟葉などひかりいでしは\n\nひがしなる山彙の上に\n...,3
341,\n\n洪積の台のはてなる\n\n一ひらの赤き粘土地\n\n\n桐の群白くひかれど\n\n枝...,3
196,\n\n\n\n（わが陋屋には、六坪ほどの庭があるのだ。愚妻は、ここに、秩序も無く何やらかや...,1
246,\n\n\n\n さっきから松原を通ってるんだが、松原と云うものは絵で見たよりもよっぽど長い...,2
60,\n\n\n\n 一\n\n 小野の小町、几帳の陰に草紙を読んでいる。そこへ突...,0
155,\n\n\n\n 芸術家というものは、つくづく困った種族である。鳥籠一つを、必死にかかえて、...,1
261,\n\n\n\n 私はこの大阪で講演をやるのは初めてであります。またこういう大勢の前に立つの...,2
141,\n\n\n\n 一人の遊蕩の子を描写して在るゆえを以て、その小説を、デカダン小説と呼ぶのは...,1
214,\n\n\n\n「煤煙」が朝日新聞に出て有名になつてから後間もなくの話であるが、著者は夫を単...,2


In [12]:
# Mecabの環境が必要です
# Mecabが動くかのテスト
import MeCab
mecab = MeCab.Tagger("-Owakati")
text = mecab.parse ("今日の中野の天気は晴れです。")
print(text)

今日 の 中野 の 天気 は 晴れ です 。 



In [13]:
# Mecabを使って、日本語を形態素解析する関数を定義
# Mecabの環境が必要です
# http://taku910.github.io/mecab/#download
import MeCab

def preprocessor(text):
    mecab = MeCab.Tagger("-Owakati")
    text=re.sub("\\n","",text)
    return mecab.parse(text)

In [14]:
# すべてのドキュメントに関数を使う
df['ドキュメント'] = df['ドキュメント'].apply(preprocessor)

In [15]:
df.head(10)

,ドキュメント,作者ID
132,はし が き もの 思う 葦 という 題名 にて 、 日本 浪 曼派 の 機関 雑誌 に...,1
309,この みち の 醸す が ごとく 粟 葉 など ひかり い でし は ひがし なる 山 彙 ...,3
341,洪 積 の 台 の はて なる 一 ひ ら の 赤き 粘土 地 桐 の 群 白く ひか れ ...,3
196,（ わが 陋屋 に は 、 六 坪 ほど の 庭 が ある の だ 。 愚妻 は 、 ここ ...,1
246,さっき から 松原 を 通っ てる ん だ が 、 松原 と 云う もの は 絵 で 見...,2
60,一 小野 の 小町 、 几帳 の 陰 に 草紙 を 読ん...,0
155,芸術 家 という もの は 、 つくづく 困っ た 種族 で ある 。 鳥 籠 一つ を...,1
261,私 は この 大阪 で 講演 を やる の は 初めて で あり ます 。 また こうい...,2
141,一 人 の 遊蕩 の 子 を 描写 し て 在る ゆえ を以て 、 その 小説 を 、 ...,1
214,「 煤煙 」 が 朝日新聞 に 出 て 有名 に な つて から 後 間もなく の 話 で ...,2


In [16]:
# 日本語のstopを定義
# 出典：https://qiita.com/chamao/items/7edaba62b120a660657e
def download_stopwords(path):
    url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
    if os.path.exists(path):
        print('File already exists.')
    else:
        print('Downloading...')
        # Download the file from `url` and save it locally under `file_name`:
        urllib.request.urlretrieve(url, path)

def create_stopwords(file_path):
    stop_words = []
    for w in open(path, "r",encoding='utf-8'):
        w = w.replace('\n','')
        if len(w) > 0:
            stop_words.append(w)
    return stop_words    

path = "stop_words.txt"
download_stopwords(path)
stop = create_stopwords(path)

File already exists.


In [17]:
# trainとtestデータに分ける
X_train = df.loc[:300, 'ドキュメント'].values
y_train = df.loc[:300, '作者ID'].values
X_test = df.loc[300:, 'ドキュメント'].values
y_test = df.loc[300:, '作者ID'].values

In [39]:
def tokenizer(text):
    return text.split()

# 名詞と形容詞のだけに絞り、形容詞は原形にする
def tokenizer_porter(text):
    mecab = MeCab.Tagger("-Ochasen")
    text=mecab.parseToNode(text)
    newText = []
    while text:
        featureSplit = text.feature.split(",")
        if featureSplit[0]=="名詞" or featureSplit[0]=="形容詞":
            newText.append(featureSplit[6])
        text=text.next
    return newText

print(tokenizer_porter("今日 の 中野 の 天気 は 美しく 晴れ です 。 "))

['今日', '中野', '天気', '美しい']


In [42]:
# 5分割交差検定でロジスティック回帰モデルの最適なパラメータ集合を求める

# 日本語のtfidfの参照
# https://analytics-note.xyz/machine-learning/bow-tfidf-one-character/

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None,
                       token_pattern='(?u)\\b\\w+\\b',
                       max_features=5000)

# グリッドサーチを定義
param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

# pipelineに入れる
lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

# 注意！！gs_lr_tfidfのn_jobs=-1でマルチプロセス処理を最大で実行。
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

In [43]:
# この処理は数分かかる恐れがあります!
# トレーニング開始
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  4.3min finished
C:\Users\gooda\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\gooda\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\gooda\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...e, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'vect__ngram_range': [(1, 1)], 'vect__stop_words': [['あそこ', 'あたり', 'あちら', 'あっち', 'あと', 'あな', 'あなた', 'あれ', 'いくつ', 'いつ', 'いま', 'いや', 'いろいろ', 'うち', 'おおまか', 'おまえ', 'おれ', 'がい', 'かく', 'かたち', 'かやの', 'から', 'がら', 'きた', 'くせ', 'ここ', 'こっち', 'こと', 'ごと', 'こちら', 'ごっちゃ', 'これ', 'これら', 'ごろ', 'さまざま', 'さらい...kenizer_porter at 0x000001C6BE2C20D0>], 'clf__penalty': ['l1', 'l2'], 'clf__C': [1.0, 10.0, 100.0]}],
       pre_dispatch='2*n_jobs', refit=T

In [44]:
# ベストなパラメーター集合とそのスコア
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

Best parameter set: {'clf__C': 100.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x000001C6BE2C2598>} 
CV Accuracy: 0.859


In [45]:
# 90%程度の正解率で肯定的か否定的かを判定できる
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

Test Accuracy: 0.877
